# Before your start:
- Read the README.md file
- Comment as much as you can and use the resources (README.md file)
- Happy learning!

In [1]:
import numpy as np
import pandas as pd

# Challenge 1 - Exploring the Data

In this challenge, we will examine all salaries of employees of the City of Chicago. We will start by loading the dataset and examining its contents

In [2]:
salaries = pd.read_csv('../Current_Employee_Names__Salaries__and_Position_Titles.csv')

Examine the `salaries` dataset using the `head` function below.

In [3]:
salaries.head()

,Name,Job Titles,Department,Full or Part-Time,Salary or Hourly,Typical Hours,Annual Salary,Hourly Rate
0,"AARON, JEFFERY M",SERGEANT,POLICE,F,Salary,NaN,101442.0,NaN
1,"AARON, KARINA",POLICE OFFICER (ASSIGNED AS DETECTIVE),POLICE,F,Salary,NaN,94122.0,NaN
2,"AARON, KIMBERLEI R",CHIEF CONTRACT EXPEDITER,GENERAL SERVICES,F,Salary,NaN,101592.0,NaN
3,"ABAD JR, VICENTE M",CIVIL ENGINEER IV,WATER MGMNT,F,Salary,NaN,110064.0,NaN
4,"ABASCAL, REECE E",TRAFFIC CONTROL AIDE-HOURLY,OEMC,P,Hourly,20.0,NaN,19.86


We see from looking at the `head` function that there is quite a bit of missing data. Let's examine how much missing data is in each column. Produce this output in the cell below

In [4]:
def nancols(x):
    nan_cols = x.isna().sum()
    return nan_cols[nan_cols>0].sort_values(ascending=False)

In [5]:
nancols(salaries)

Typical Hours    25161
Hourly Rate      25161
Annual Salary     8022
dtype: int64

Let's also look at the count of hourly vs. salaried employees. Write the code in the cell below

In [6]:
def col_unique_counts(df, col):
    counts = df[col].value_counts().reset_index()
    counts.columns = [col, 'Count']
    return counts

In [7]:
col_unique_counts(salaries,'Salary or Hourly')

,Salary or Hourly,Count
0,Salary,25161
1,Hourly,8022


What this information indicates is that the table contains information about two types of employees - salaried and hourly. Some columns apply only to one type of employee while other columns only apply to another kind. This is why there are so many missing values. Therefore, we will not do anything to handle the missing values.

There are different departments in the city. List all departments and the count of employees in each department.

In [10]:
salaries.groupby('Department')['Name'].count()


Department
ADMIN HEARNG                39
ANIMAL CONTRL               81
AVIATION                  1629
BOARD OF ELECTION          107
BOARD OF ETHICS              8
BUDGET & MGMT               46
BUILDINGS                  269
BUSINESS AFFAIRS           171
CITY CLERK                  84
CITY COUNCIL               411
COMMUNITY DEVELOPMENT      207
COPA                       116
CULTURAL AFFAIRS            65
DISABILITIES                28
DoIT                        99
FAMILY & SUPPORT           615
FINANCE                    560
FIRE                      4641
GENERAL SERVICES           980
HEALTH                     488
HUMAN RELATIONS             16
HUMAN RESOURCES             79
INSPECTOR GEN               87
LAW                        407
LICENSE APPL COMM            1
MAYOR'S OFFICE              85
OEMC                      2102
POLICE                   13414
POLICE BOARD                 2
PROCUREMENT                 92
PUBLIC LIBRARY            1015
STREETS & SAN             21

# Challenge 2 - Hypothesis Tests

In this section of the lab, we will test whether the hourly wage of all hourly workers is significantly different from $30/hr. Import the correct one sample test function from scipy and perform the hypothesis test for a 95% two sided confidence interval.

In [14]:
import numpy as np

from scipy.stats import t

from scipy.stats import ttest_1samp

In [30]:
mu = 30
umbral= 0.05
hourly= salaries.loc[salaries['Salary or Hourly'] == 'Hourly', 'Hourly Rate']


In [27]:
stat, p_value=ttest_1samp(hourly, mu, alternative='two-sided')

In [28]:
p_value #practicamente cero asi que hipothesis H0 podemos rechazarla. 

4.3230240486229894e-92

In [31]:
p_value<umbral

True

We are also curious about salaries in the police force. The chief of police in Chicago claimed in a press briefing that salaries this year are higher than last year's mean of $86000/year a year for all salaried employees. Test this one sided hypothesis using a 95% confidence interval.

Hint: A one tailed test has a p-value that is half of the two tailed p-value. If our hypothesis is greater than, then to reject, the test statistic must also be positive.

In [39]:
police_salaries = salaries.loc[(salaries['Salary or Hourly'] == 'Salary') & (salaries['Department'] == 'POLICE'), 'Annual Salary']

In [53]:
# H0 salarios menores q 860000
# H1 salarios mayores q 860000

mu=86000
police_salaries
stat, p_value=ttest_1samp(police_salaries, mu, alternative='greater') 

In [54]:
stat, p_value #H0 por lo tanto se puede refutar. 

(3.081997005712994, 0.0010301701775482569)

In [55]:
police_salaries.mean() #double-check

86486.41450313339

In [51]:
#a la inversa
# H0 salarios mayores q 860000
# H1 salarios menores q 860000

mu=86000
police_salaries
stat, p_value=ttest_1samp(police_salaries, mu, alternative='less')

In [52]:
stat, p_value #H1 no se sustenta.

(3.081997005712994, 0.9989698298224517)

Using the `crosstab` function, find the department that has the most hourly workers. 

In [61]:
#cross_tab = pd.crosstab(salaries['Department'], salaries['Salary or Hourly'] == 'Hourly')

In [66]:
dept_group = salaries.loc[salaries['Salary or Hourly'] == 'Hourly'].groupby('Department').agg(count=('Department', 'count')).sort_values(by='count', ascending=False)

dept_group

,count
Department,
STREETS & SAN,1862
WATER MGMNT,1513
OEMC,1273
AVIATION,1082
GENERAL SERVICES,765
TRANSPORTN,725
PUBLIC LIBRARY,299
FAMILY & SUPPORT,287
CITY COUNCIL,64


The workers from the department with the most hourly workers have complained that their hourly wage is less than $35/hour. Using a one sample t-test, test this one-sided hypothesis at the 95% confidence level.

In [71]:
#H0 mayor que 35
#H1 menor que 35

mu = 35
umbral= 0.05
hourly_SS= salaries.loc[(salaries['Salary or Hourly'] == 'Hourly') & (salaries['Department'] == 'STREETS & SAN'),'Hourly Rate']

stat, p_value= ttest_1samp(hourly_SS, mu, alternative='less')

p_value<umbral #H0 se puede refutar o no hay suficientes indicios para apoyarlo. Por lo tanto los trabajadores estarian en lo cierto 

True

# Challenge 3 - Constructing Confidence Intervals

While testing our hypothesis is a great way to gather empirical evidence for accepting or rejecting the hypothesis, another way to gather evidence is by creating a confidence interval. A confidence interval gives us information about the true mean of the population. So for a 95% confidence interval, we are 95% sure that the mean of the population is within the confidence interval. 
).

To read more about confidence intervals, click [here](https://en.wikipedia.org/wiki/Confidence_interval).


In the cell below, we will construct a 95% confidence interval for the mean hourly wage of all hourly workers. 

The confidence interval is computed in SciPy using the `t.interval` function. You can read more about this function [here](https://docs.scipy.org/doc/scipy-0.14.0/reference/generated/scipy.stats.t.html).

To compute the confidence interval of the hourly wage, use the 0.95 for the confidence level, number of rows - 1 for degrees of freedom, the mean of the sample for the location parameter and the standard error for the scale. The standard error can be computed using [this](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.sem.html) function in SciPy.

In [72]:
from scipy.stats import sem
from scipy.stats import t

In [77]:
# 95% de confianza de que la media esté en ese intervalo
df=salaries['Hourly Rate'].dropna() #esto para que no nos tome los nulos y a la hora de hacer el len para calcular n-1 tome las 33k filas en vez de las 8k correspondientes de los hourlies
t.interval(0.95,     #alpha aka intervalo de confianza
           len(df)-1, #n-1, grados de independencia porque se trata de un sample no población
           loc=df.mean(), #media en cuestión
           scale=sem(df) #error standard
           )

(32.52345834488425, 33.05365708767623)

In [80]:
df.mean() #con esto podemos verificar que efectivamente la media se encuentra dentro del intervalo de confianza que hemos creado.

32.78855771628024

Now construct the 95% confidence interval for all salaried employeed in the police in the cell below.

In [85]:
police_salaries.dropna()
t.interval(0.95,
           len(police_salaries)-1,
           loc=police_salaries.mean(),
           scale=sem(police_salaries)
           )

(86177.05631531784, 86795.77269094894)

In [86]:
police_salaries.mean()

86486.41450313339

# Challenge 4 - Hypothesis Tests of Proportions

Another type of one sample test is a hypothesis test of proportions. In this test, we examine whether the proportion of a group in our sample is significantly different than a fraction. 

You can read more about one sample proportion tests [here](http://sphweb.bumc.bu.edu/otlt/MPH-Modules/BS/SAS/SAS6-CategoricalData/SAS6-CategoricalData2.html).

In the cell below, use the `proportions_ztest` function from `statsmodels` to perform a hypothesis test that will determine whether the number of hourly workers in the City of Chicago is significantly different from 25% at the 95% confidence level.

In [89]:
#%pip install statsmodels

In [90]:
from statsmodels.stats.proportion import proportions_ztest


In [92]:
z_score, p_value= proportions_ztest(salaries['Salary or Hourly'].value_counts()[1], #valores unicos de hourlies, los 8000 y cacho
                                    len(salaries), #el total de elementos 
                                    0.25) #proporcion 
z_score, p_value #z_score  es el numero de std y p_value la probabilidad de la hipothesis 

(-3.5099964213703005, 0.0004481127249057967)

In [94]:
p_value<umbral #entonces signigica que es diferente a 25%

True

In [93]:
#salaries['Salary or Hourly'].value_counts() esto nos devuelve conteo unicos de esta columna
#en este caso son 2 : salary/hourly en este orden. 
#como nos interesa los hourlies le ponemos indice 1.

#en verdad esto es una movie mas facil hacer un  len(salaries[salaries['hour/sal]=='hourly])

Salary or Hourly
Salary    25161
Hourly     8022
Name: count, dtype: int64